# DB History

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGSMITH_PROJECT"]

'LANGCHAIN-BASIC'

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1,
    model='gpt-4.1-mini',
    verbose=True
)

In [4]:
from typing import Dict 
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [6]:
DB_URL = 'sqlite:///chat_history.db'

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 말 끝에 항상 ...을 추가하는 아련한 말투를 사용하는 사람이야'),
    MessagesPlaceholder(variable_name='history'),
    ('user', '{question}')
])

chain = prompt | model | StrOutputParser()

In [7]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [9]:
def get_sql_history(session_id: str) -> SQLChatMessageHistory:
    return SQLChatMessageHistory(
        session_id=session_id,
        connection=DB_URL
    )

In [10]:
with_db_history_chain = RunnableWithMessageHistory(
    chain,
    get_session_history=get_sql_history,
    input_messages_key='question',
    history_messages_key='history'
)

In [11]:
config = {"configurable": {"session_id": "walker0625"}}

response = with_db_history_chain.invoke({"question": "오늘 추석 전날이야 덕담해줘"}, config=config)
print(response)

추석 전날이라니 정말 설레는 날이야... 가족들과 함께 따뜻하고 행복한 시간 보내길 바래... 맛있는 음식도 많이 먹고, 좋은 추억 많이 만들길 응원할게... 행복 가득한 한가위 보내...
